# Assignment 2:
# Mining Frequent Patterns

In this assignment, we will using Apriori algorithm to mine the frequent patterns (sort terms, just set k = 3, 4, 5) from a social network dataset.

##Data Preparation

At first, download the dataset from http://snap.stanford.edu/data/gemsec_facebook_dataset.tar.gz and read the dataset description. Then upload and connect it to your VM.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In this pratice, we just use the first datafile (artist_edges.csv).

Now load the data to a variable in python.

In [ ]:
# !pip install ordered-set

labdir = "/content/drive/MyDrive/Colab Notebooks/Assignment/Assignment2/data/facebook_clean_data/"
import csv

with open(labdir+"artist_edges.csv") as csvfile:
    facebook_clean_data = list(csv.reader(csvfile))


##Apriori Algorithm

Apriori is an algorithm for frequent item set mining and association rule learning over relational databases.

Before to perform Apriori, we need to define a function 'data_processing' for data preprocessing firstly.

'data_set' is a list of transactions. Each transaction contains several items.

In [ ]:
def processing_data():
  dic = {}
  for i in facebook_clean_data[1:]:
    if not i[0] in dic:
      dic[i[0]] = [i[1]]
    else:
      dic[i[0]].append(i[1])
  data_set = list(dic.values())
  #print(data_set)
  return data_set
# data_set = processing_data()

Now define a function 'create_C1' ot create frequent candidate 1-itemset C1 by scaning data set.

C1: A set which contains all frequent candidate 1-itemsets.

In [ ]:
def create_C1(data_set):
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1


Judge whether a frequent candidate k-itemset satisfy Apriori property.

Ck_item: A frequent candidate k-itemset in Ck which contains all frequent candidate k-itemsets.

Lksub1: A set which contains all frequent candidate (k-1)-itemsets.

In [ ]:
def is_apriori(Ck_item, Lksub1):
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True

Create Ck, a set which contains all all frequent candidate k-itemsets
    by Lk-1's own connection operation.

k: the item number of a frequent itemset.

Ck: A set which contains all all frequent candidate k-itemsets.

In [ ]:
def create_Ck(Lksub1, k):
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)
    list_Lksub1 = sorted(list_Lksub1, key=lambda x: sorted(x))
    for i in range(len_Lksub1):
        # print("-"*15)
        for j in range(i+1, len_Lksub1):
            # print("-"*10)
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()
            l2.sort()

            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # pruning
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck

Generate Lk by executing a delete policy from Ck.

min_support: The minimum support.

support_data: A dictionary. The key is frequent itemset and the value is support.

Lk: A set which contains all all frequent k-itemsets.

In [ ]:
from os import times_result
def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    Lk = set()
    item_count = {}

    for t in data_set:
        for item in Ck:
            if item.issubset(t):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1
    t_num = float(len(data_set))

    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk

In [ ]:
import math
def get_best_min_support_for_L1(data_set, C1):
    item_count = {}
    for t in data_set:
      for item in C1:
          if item.issubset(t):
              if item not in item_count:
                  item_count[item] = 1
              else:
                  item_count[item] += 1
    t_num = float(len(data_set))
    suitable_min_support = max(item_count.values())/t_num
    n = 2
    suitable_min_support = math.floor(suitable_min_support*10**n - 1)/(10**n)
    return suitable_min_support

Generate all frequent itemsets.

k: Maximum number of items for all frequent itemsets.

L: The list of Lk.

In [ ]:
def generate_L(data_set, k, min_support):
    support_data = {}
    C1 = create_C1(data_set)
    #initialize the min_support at L1
    suitable_min_support = get_best_min_support_for_L1(data_set, C1)
    if(suitable_min_support < min_support):
        min_support = suitable_min_support
    print("min_support ",min_support,"on running")

    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    Lksub1 = L1.copy()
    L = []
    L.append(Lksub1)
    i = 2
    for i in range(2, k+1):
        Ci = create_Ck(Lksub1, i)
        if(len(Ci)==0):
          break
        Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
        Lksub1 = Li.copy()
        L.append(Lksub1)
        if(len(Lksub1) < 2):
          break
    print("-"*10)
    print(L)
    print("-" * 10)
    print("support_data:",support_data)
    return L, support_data

Generate big rules from frequent itemsets.

min_conf: Minimal confidence.

big_rule_list: A list which contains all big rules. Each big rule is represented as a 3-tuple.

In [ ]:
def generate_big_rules(L, support_data, min_conf):
    big_rule_list = []
    sub_set_list = []
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:
                if sub_set.issubset(freq_set):
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= min_conf and big_rule not in big_rule_list:
                        big_rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    return big_rule_list

In [ ]:
data_set = processing_data()
# data_set = [['l1', 'l2', 'l5'], ['l2', 'l4'], ['l2', 'l3'], ['l1', 'l2', 'l4'], ['l1', 'l3'], ['l2', 'l3'],
#            ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2', 'l3']]

L, support_data = generate_L(data_set,k=3, min_support=0.2)
big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)
print(len(L))
print(L[0])
for Lk in L:
  print("="*50)
  print("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
  print("="*50)
  for freq_set in Lk:
    print(freq_set, support_data[freq_set])
print("Big Rules")
for item in big_rules_list:
  print(item[0], "=>", item[1], "conf: ", item[2])

min_support  0.01 on running
----------
[{frozenset({'37078'}), frozenset({'18540'}), frozenset({'26345'}), frozenset({'32478'}), frozenset({'29570'}), frozenset({'39940'}), frozenset({'49956'}), frozenset({'46074'}), frozenset({'37439'}), frozenset({'37638'}), frozenset({'45196'}), frozenset({'40251'}), frozenset({'48438'}), frozenset({'32660'}), frozenset({'46117'}), frozenset({'44960'}), frozenset({'22486'}), frozenset({'32294'}), frozenset({'38888'}), frozenset({'9187'}), frozenset({'42543'}), frozenset({'42892'}), frozenset({'44169'}), frozenset({'24170'}), frozenset({'50001'}), frozenset({'44657'}), frozenset({'39019'}), frozenset({'48794'}), frozenset({'31456'}), frozenset({'41998'}), frozenset({'48099'}), frozenset({'46775'}), frozenset({'49209'}), frozenset({'39158'}), frozenset({'45197'}), frozenset({'39379'}), frozenset({'41361'}), frozenset({'40666'}), frozenset({'11204'}), frozenset({'37329'}), frozenset({'39284'}), frozenset({'40958'}), frozenset({'47790'}), frozenset({'3